In [202]:
import re
import pandas as pd

In [203]:
# **Домашнее задание к лекции.
# Задание 1**
# Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
# - оценка 2 и меньше - низкий рейтинг
# - оценка 4 и меньше - средний рейтинг
# - оценка 4.5 и 5 - высокий рейтинг
# Результат классификации запишите в столбец class

In [204]:
df = pd.read_csv('ratings.csv')
df['class'] = df.apply((lambda line: 'low_rating' if line['rating'] <= 2 else ('middle_rating' if line['rating'] <= 4 else 'high_rating')), axis=1)
df


,userId,movieId,rating,timestamp,class
0,1,31,2.5,1260759144,middle_rating
1,1,1029,3.0,1260759179,middle_rating
2,1,1061,3.0,1260759182,middle_rating
3,1,1129,2.0,1260759185,low_rating
4,1,1172,4.0,1260759205,middle_rating
...,...,...,...,...,...
99999,671,6268,2.5,1065579370,middle_rating
100000,671,6269,4.0,1065149201,middle_rating
100001,671,6365,4.0,1070940363,middle_rating
100002,671,6385,2.5,1070979663,middle_rating


In [205]:
# Задание 2
# Используем файл keywords.csv.
# Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. 
# Т.е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. 
# Если поисковый запрос не содержит названия города, то ставим ‘undefined’.
# Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
# geo_data = {
# 'Центр': ['москва', 'тула', 'ярославль'],
# 'Северо-Запад': ['петербург', 'псков', 'мурманск'],
# 'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
# }
# Результат классификации запишите в отдельный столбец region.

In [206]:
geo_data = {'Центр': ['москва', 'тула', 'ярославль'], 
            'Северо-Запад': ['петербург', 'псков', 'мурманск'], 
            'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']}

In [207]:
def isCity(string, geo_data):
    region = [key for key, value in geo_data.items() if [item for item in value if item.lower() in string.lower()]]
    return ", ".join(region) if region else 'undefined'
df = pd.read_csv('keywords.csv')
df['region'] = df['keyword'].apply(isCity, geo_data=geo_data)
df.sort_values('region', ascending=False)

,keyword,shows,region
52463,москва санкт петербург,9756,"Центр, Северо-Запад"
48830,сапсан москва санкт петербург,10469,"Центр, Северо-Запад"
56895,авито москва недвижимость,10611,Центр
73226,фанатик спартак москва,5353,Центр
68005,гостиница москва,5651,Центр
...,...,...,...
33455,тобот мультфильм на русском смотреть онлайн,9790,undefined
33454,логические загадки,10369,undefined
33453,пенсионный фонд рф,9875,undefined
33452,olesya fattakhova,9832,undefined


In [208]:
# Задание 3 (бонусное)
# Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. 
# Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия 
# (файл ratings.csv из базы https://grouplens.org/datasets/movielens). 
# Т.е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.
# При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:
# В переменную years запишите список из всех годов с 1950 по 2010.
# Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. 
# Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
# для каждой строки пройдите по всем годам списка years
# если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
# если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
# Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’
# Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [209]:
def getYear(string):
    year = re.search(r'(\d{4})', string)
    return int(year.group(0)) if year else 1900
df_movies = pd.read_csv('movies.csv')
df_movies['year'] = df_movies['title'].apply(getYear)
df_movies = df_movies[df_movies['year'].between(1950, 2010)].set_index('movieId').join(pd.read_csv('ratings.csv').set_index('movieId'), on='movieId', how='left')[['year', 'rating']].groupby('year').agg({'rating': 'mean'}).round({'rating': 1}).sort_values('rating', ascending=False)
df_movies

,rating
year,
1962,4.0
1957,4.0
1972,4.0
1951,4.0
1974,4.0
...,...
1996,3.4
2000,3.4
2003,3.4
